# Stock Prices for Companies After a Breach

Stock prices typically fall right after a breach. Many people claim that a data breach would make them less likely to trust a company again. The analysis looks at stock prices for a company before and after a breach to determine whether there was an impact, or if investors/consumers forgave the company.

Companies under analysis (these companies reported breaches in 2018 and 2019)

* Marriott
* Capital One
* Under Armour
* First American
* Facebook

In [1]:
#import statements
import pandas as pd
import re
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns

The dates were generated by looking at the date of the breach for each company, and then calculating 1 year prior, 9 months prior, 6 months prior, 3 months prior, 1 month prior, the day before, the day of, the day after, 1 month after, 3 months after, 6 months after, 9 months after, and a year later.

In [2]:
#prepare dates that are relevant for the analysis.
# read in nasdaq data

rel_dates=['2018-07-30','2018-11-02','2019-01-31','2019-05-01','2019-07-01','2019-07-23','2019-07-29',
           '2019-07-30','2019-07-31','2019-08-06','2019-08-29','2019-10-28','2020-01-27','2020-04-27',
           '2020-07-29','2018-05-24','2018-08-27','2018-11-26','2019-02-22','2019-04-24','2019-05-17',
           '2019-05-23','2019-05-24','2019-05-27','2019-05-31','2019-06-24','2019-08-22','2019-11-20',
           '2020-02-18','2020-05-25','2018-04-03','2018-07-06','2018-10-05','2019-01-03','2019-03-04',
           '2019-03-27','2019-04-02','2019-04-03','2019-04-04','2019-04-10','2019-05-03','2019-07-02',
           '2019-09-30','2019-12-30','2020-04-02','2017-11-30','2018-03-05','2018-06-04','2018-08-31',
           '2018-10-31','2018-11-23','2018-11-29','2018-11-30','2018-12-03','2018-12-07','2018-12-31',
           '2019-02-28','2019-05-29','2019-08-27','2019-12-02','2017-03-29','2017-07-05','2017-09-29',
           '2017-12-29','2018-02-27','2018-03-22','2018-03-28','2018-03-29','2018-03-30','2018-04-05',
           '2018-04-30','2018-06-27','2018-09-25','2018-12-24','2019-03-29']
nasdaq = pd.read_csv("/home/fyr/lede/story/cleanfinal/analysis/data/IXIC.csv")
nasdaq.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-12-15,4679.669922,4690.580078,4592.430176,4605.160156,4605.160156,2143610000
1,2014-12-16,4572.790039,4645.189941,4547.310059,4547.830078,4547.830078,2231670000
2,2014-12-17,4556.899902,4651.899902,4550.700195,4644.310059,4644.310059,2279930000
3,2014-12-18,4712.390137,4748.399902,4697.089844,4748.399902,4748.399902,2172260000
4,2014-12-19,4752.600098,4782.129883,4738.290039,4765.379883,4765.379883,3287920000
5,2014-12-22,4759.040039,4781.930176,4757.799805,4781.419922,4781.419922,1720070000
6,2014-12-23,4798.029785,4798.060059,4761.390137,4765.419922,4765.419922,1590820000
7,2014-12-24,4770.120117,4787.569824,4768.669922,4773.470215,4773.470215,729750000
8,2014-12-26,4788.060059,4814.950195,4787.850098,4806.859863,4806.859863,930220000
9,2014-12-29,4801.259766,4813.720215,4798.899902,4806.910156,4806.910156,1227740000


In [3]:
#filter out only nasdaq data that is relevant to the time period
rslt = nasdaq.loc[nasdaq['Date'].isin(rel_dates)] 
rslt

,Date,Open,High,Low,Close,Adj Close,Volume
575,2017-03-29,5875.350098,5900.870117,5870.930176,5897.549805,5897.549805,1718680000
642,2017-07-05,6122.060059,6163.620117,6100.419922,6150.859863,6150.859863,1872130000
703,2017-09-29,6461.279785,6497.979980,6454.859863,6495.959961,6495.959961,1967720000
746,2017-11-30,6852.799805,6888.649902,6838.479980,6873.970215,6873.970215,2458580000
766,2017-12-29,6952.609863,6954.979980,6903.390137,6903.390137,6903.390137,1571730000
...,...,...,...,...,...,...,...
1184,2019-08-29,7945.779785,7992.290039,7925.830078,7973.390137,7973.390137,1700840000
1205,2019-09-30,7964.089844,8012.160156,7949.629883,7999.339844,7999.339844,1805820000
1225,2019-10-28,8285.769531,8335.559570,8285.269531,8325.990234,8325.990234,1956280000
1242,2019-11-20,8543.570313,8578.269531,8468.629883,8526.730469,8526.730469,2596030000


In [27]:
row0=['2018-07-30','2018-05-24','2018-04-03','2017-11-30','2017-03-29']
row1=['2018-11-02','2018-08-27','2018-07-06','2018-03-05','2017-07-05']
row2=['2019-01-31','2018-11-26','2018-10-05','2018-06-04','2017-09-29']
row3=['2019-05-01','2019-02-22','2019-01-03','2018-08-31','2017-12-29']
row4=['2019-07-01','2019-04-24','2019-03-04','2018-10-31','2018-02-27']
row5=['2019-07-23','2019-05-17','2019-03-27','2018-11-23','2018-03-22']
row6=['2019-07-29','2019-05-23','2019-04-02','2018-11-29','2018-03-28']
row7=['2019-07-30','2019-05-24','2019-04-03','2018-11-30','2018-03-29']
row8=['2019-07-31','2019-05-27','2019-04-04','2018-12-03','2018-03-30']
row9=['2019-08-06','2019-05-31','2019-04-10','2018-12-07','2018-04-05']
row10=['2019-08-29','2019-06-24','2019-05-03','2018-12-31','2018-04-30']
row11=['2019-10-28','2019-08-22','2019-07-02','2019-02-28','2018-06-27']
row12=['2020-01-27','2019-11-20','2019-09-30','2019-05-29','2018-09-25']
row13=['2020-04-27','2020-02-18','2019-12-30','2019-08-27','2018-12-24']
row14=['2020-07-29','2020-05-25','2020-04-02','2019-12-02','2019-03-29']

cal_means=[]
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row0)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row1)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row2)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row3)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row4)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row5)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row6)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row7)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row8)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row9)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row10)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row11)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row12)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row13)].Open.mean())
cal_means.append(nasdaq.loc[nasdaq['Date'].isin(row14)].Open.mean())
cal_means

nq = pd.DataFrame(cal_means, columns=['nasdaq'])
nq

,nasdaq
0,6961.952051
1,7270.908106
2,7228.157910
3,7446.250000
4,7719.628027
5,7590.129883
6,7611.219922
7,7612.495898
8,7890.396485
9,7492.186035


In [4]:
# read in CapitalOne data
ncof_dates=['2018-07-30','2018-11-02','2019-01-31','2019-05-01','2019-07-01','2019-07-23','2019-07-29',
            '2019-07-30','2019-07-31','2019-08-06','2019-08-29','2019-10-28','2020-01-27','2020-04-27',
            '2020-07-29']
cof = pd.read_csv("/home/fyr/lede/story/cleanfinal/analysis/data/COF.csv")
cof.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-12-15,80.510002,80.510002,78.580002,79.610001,71.216408,4607400
1,2014-12-16,78.870003,79.940002,77.720001,78.230003,69.981895,3887600
2,2014-12-17,78.419998,79.620003,78.150002,79.610001,71.216408,5072400
3,2014-12-18,80.339996,82.169998,80.330002,82.160004,73.497551,4340300
4,2014-12-19,82.459999,82.940002,81.720001,82.430000,73.739098,4057100
5,2014-12-22,82.400002,82.930000,82.080002,82.889999,74.150581,2040800
6,2014-12-23,83.389999,83.769997,83.209999,83.279999,74.499474,1685200
7,2014-12-24,83.570000,83.570000,83.190002,83.230003,74.454750,1040600
8,2014-12-26,83.379997,83.779999,83.180000,83.209999,74.436836,957600
9,2014-12-29,82.959999,83.959999,82.830002,83.300003,74.517380,1601200


In [5]:
#filter to have only Capital One data that fits the range
rsltc = cof.loc[cof['Date'].isin(ncof_dates)] 
rsltc = rsltc.reset_index()
rsltc = rsltc[['Open','High']]
rsltc = rsltc.rename(columns={'Open': "CO_Open", 'High': "CO_High"})
rsltc

,CO_Open,CO_High
0,94.500000,95.849998
1,89.300003,89.849998
2,80.279999,80.809998
3,92.559998,93.489998
4,92.300003,92.830002
5,91.930000,94.500000
6,98.029999,98.279999
7,91.480003,92.000000
8,91.300003,93.410004
9,87.150002,88.419998


In [6]:
# read in Marriott data
nmar_dates=['2017-11-30','2018-03-05','2018-06-04','2018-08-31','2018-10-31','2018-11-23','2018-11-29',
            '2018-11-30','2018-12-03','2018-12-07','2018-12-31','2019-02-28','2019-05-29','2019-08-27',
            '2019-12-02']
mar = pd.read_csv("/home/fyr/lede/story/cleanfinal/analysis/data/MAR.csv")
mar.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-12-15,77.500000,78.290001,76.269997,77.570000,72.498421,2640100
1,2014-12-16,77.839996,77.930000,76.220001,76.279999,71.292747,3311600
2,2014-12-17,76.290001,77.080002,75.620003,76.739998,71.722687,2353700
3,2014-12-18,77.639999,77.879997,76.910004,77.349998,72.292809,2353400
4,2014-12-19,77.330002,77.970001,76.980003,77.430000,72.367569,4492400
5,2014-12-22,77.410004,78.610001,77.349998,78.459999,73.330223,1257100
6,2014-12-23,78.750000,79.029999,78.339996,78.540001,73.404991,1577000
7,2014-12-24,78.089996,78.570000,77.860001,78.349998,73.227409,736900
8,2014-12-26,78.669998,78.989998,78.519997,78.690002,73.545197,891500
9,2014-12-29,78.720001,79.250000,78.160004,79.080002,73.909683,1115100


In [7]:
#filter to have only data that fits the range

rsltm = mar.loc[mar['Date'].isin(nmar_dates)] 
rsltm = rsltm.reset_index()
rsltm = rsltm[['Open','High']]
rsltm = rsltm.rename(columns={'Open': "M_Open", 'High': "M_High"})
rsltm

,M_Open,M_High
0,126.139999,127.970001
1,138.449997,138.960007
2,138.820007,138.990005
3,126.339996,127.269997
4,116.330002,118.720001
5,116.510002,118.000000
6,121.370003,122.830002
7,113.599998,116.720001
8,116.000000,120.089996
9,114.849998,116.250000


In [8]:
#read in Under Armour 
nua_dates=['2017-03-29','2017-07-05','2017-09-29','2017-12-29','2018-02-27','2018-03-22','2018-03-28',
           '2018-03-29','2018-03-30','2018-04-05','2018-04-30','2018-06-27','2018-09-25','2018-12-24',
           '2019-03-29']
ua = pd.read_csv("/home/fyr/lede/story/cleanfinal/analysis/data/UA.csv")
ua.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-07-01,84.260002,84.739998,83.709999,84.260002,84.260002,1607600.0
1,2015-07-02,84.489998,84.779999,83.900002,84.589996,84.589996,1051600.0
2,2015-07-06,83.989998,85.000000,83.709999,84.580002,84.580002,888600.0
3,2015-07-07,84.800003,84.949997,83.309998,84.940002,84.940002,1398700.0
4,2015-07-08,84.370003,85.129997,83.870003,84.389999,84.389999,1733100.0
5,2015-07-09,85.389999,85.800003,84.480003,84.489998,84.489998,1920000.0
6,2015-07-10,85.449997,86.400002,84.769997,86.239998,86.239998,1684200.0
7,2015-07-13,87.160004,89.459999,87.089996,89.349998,89.349998,2721800.0
8,2015-07-14,89.279999,89.300003,88.269997,88.800003,88.800003,1857400.0
9,2015-07-15,88.800003,89.000000,88.099998,88.279999,88.279999,1373400.0


In [9]:
#filter to have only data that fits the range
rsltu = ua.loc[ua['Date'].isin(nua_dates)] 
rsltu = rsltu.reset_index()
rsltu = rsltu[['Open','High']]
rsltu = rsltu.rename(columns={"Open": "UA_Open", "High": "UA_High"})
rsltu

,UA_Open,UA_High
0,18.430000,19.049999
1,20.500000,20.530001
2,15.090000,15.170000
3,14.070000,14.140000
4,15.510000,15.925000
5,13.940000,14.120000
6,14.370000,14.600000
7,14.310000,14.600000
8,15.550000,15.860000
9,15.370000,15.720000


In [11]:
#read in First American
nfa_dates=['2018-05-24','2018-08-27','2018-11-26','2019-02-22','2019-04-24','2019-05-17','2019-05-23',
           '2019-05-24','2019-05-27','2019-05-31','2019-06-24','2019-08-22','2019-11-20','2020-02-18',
           '2020-05-25']
fa = pd.read_csv("/home/fyr/lede/story/cleanfinal/analysis/data/FAF.csv")
fa.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-12-15,32.720001,32.900002,32.320000,32.549999,28.000837,950300
1,2014-12-16,31.730000,32.549999,31.500000,32.240002,27.734169,1822300
2,2014-12-17,32.299999,32.630001,32.009998,32.389999,27.863197,1410200
3,2014-12-18,32.700001,33.029999,32.459999,33.029999,28.413754,818200
4,2014-12-19,33.119999,33.270000,32.919998,33.090000,28.465374,1389700
5,2014-12-22,33.169998,33.310001,32.860001,33.240002,28.594404,537800
6,2014-12-23,33.470001,33.869999,33.119999,33.669998,28.964310,584700
7,2014-12-24,33.709999,34.340000,33.560001,34.299999,29.506262,481300
8,2014-12-26,34.419998,34.459999,34.119999,34.150002,29.377220,461000
9,2014-12-29,34.230000,34.509998,34.139999,34.259998,29.471853,489500


In [12]:
#filter to have only data that fits the range
rslta = fa.loc[fa['Date'].isin(nfa_dates)] 
rslta = rslta.reset_index()
rslta = rslta[['Open','High']]
rslta = rslta.rename(columns={"Open": "FA_Open", "High": "FA_High"})
rslta

,FA_Open,FA_High
0,54.299999,54.669998
1,56.880001,57.330002
2,46.759998,48.320000
3,51.209999,51.950001
4,55.970001,56.340000
5,54.820000,55.290001
6,55.290001,55.380001
7,55.160000,55.299999
8,51.299999,51.950001
9,55.650002,55.770000


In [13]:
#read in Facebook
nfb_dates=['2018-04-03','2018-07-06','2018-10-05','2019-01-03','2019-03-04','2019-03-27','2019-04-02',
           '2019-04-03','2019-04-04','2019-04-10','2019-05-03','2019-07-02','2019-09-30','2019-12-30',
           '2020-04-02']
fb = pd.read_csv("/home/fyr/lede/story/cleanfinal/analysis/data/FB.csv")
fb.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-12-15,78.459999,78.580002,76.559998,76.989998,76.989998,29396500
1,2014-12-16,76.190002,77.389999,74.589996,74.690002,74.690002,31554600
2,2014-12-17,75.010002,76.410004,74.900002,76.110001,76.110001,29203900
3,2014-12-18,76.889999,78.400002,76.510002,78.400002,78.400002,34222100
4,2014-12-19,78.750000,80.000000,78.330002,79.879997,79.879997,43335000
5,2014-12-22,80.080002,81.889999,80.000000,81.449997,81.449997,31395800
6,2014-12-23,82.019997,82.169998,80.400002,80.610001,80.610001,19865800
7,2014-12-24,81.029999,81.430000,80.750000,80.769997,80.769997,7685500
8,2014-12-26,81.019997,81.279999,80.510002,80.779999,80.779999,10647400
9,2014-12-29,80.489998,80.959999,79.709999,80.019997,80.019997,14134700


In [14]:
#filter to have only data that fits the range
rsltf = fb.loc[fb['Date'].isin(nfb_dates)] 
rsltf = rsltf.reset_index()
rsltf = rsltf[['Open','High']]
rsltf = rsltf.rename(columns={"Open": "FB_Open", "High": "FB_High"})
rsltf

,FB_Open,FB_High
0,156.550003,157.389999
1,198.449997,203.639999
2,159.210007,160.899994
3,134.690002,137.169998
4,163.899994,167.500000
5,167.850006,168.940002
6,170.139999,174.899994
7,174.500000,177.960007
8,176.020004,178.000000
9,178.179993,178.789993


In [28]:
result = pd.concat([rsltm, rsltu], axis=1, sort=False)
result = pd.concat([result, rsltc], axis=1, sort=False)
result = pd.concat([result, rslta], axis=1, sort=False)
result = pd.concat([result, rsltf], axis=1, sort=False)
result = pd.concat([result, nq], axis=1, sort=False)
result

,M_Open,M_High,UA_Open,UA_High,CO_Open,CO_High,FA_Open,FA_High,FB_Open,FB_High,nasdaq
0,126.139999,127.970001,18.430000,19.049999,94.500000,95.849998,54.299999,54.669998,156.550003,157.389999,6961.952051
1,138.449997,138.960007,20.500000,20.530001,89.300003,89.849998,56.880001,57.330002,198.449997,203.639999,7270.908106
2,138.820007,138.990005,15.090000,15.170000,80.279999,80.809998,46.759998,48.320000,159.210007,160.899994,7228.157910
3,126.339996,127.269997,14.070000,14.140000,92.559998,93.489998,51.209999,51.950001,134.690002,137.169998,7446.250000
4,116.330002,118.720001,15.510000,15.925000,92.300003,92.830002,55.970001,56.340000,163.899994,167.500000,7719.628027
5,116.510002,118.000000,13.940000,14.120000,91.930000,94.500000,54.820000,55.290001,167.850006,168.940002,7590.129883
6,121.370003,122.830002,14.370000,14.600000,98.029999,98.279999,55.290001,55.380001,170.139999,174.899994,7611.219922
7,113.599998,116.720001,14.310000,14.600000,91.480003,92.000000,55.160000,55.299999,174.500000,177.960007,7612.495898
8,116.000000,120.089996,15.550000,15.860000,91.300003,93.410004,51.299999,51.950001,176.020004,178.000000,7890.396485
9,114.849998,116.250000,15.370000,15.720000,87.150002,88.419998,55.650002,55.770000,178.179993,178.789993,7492.186035


In [29]:
result.to_csv('../src/data/trust.csv')